<a href="https://colab.research.google.com/github/AdrienAlvarez/Scraping_offres_emploi/blob/main/Notebook_Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.2/400.2 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.9 MB/s eta 0:00:00


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from datetime import datetime
import pandas as pd
import time
from joblib import load
import re
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')


chrome_options = Options()
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--headless")  # Si vous voulez exécuter en mode headless

driver = webdriver.Chrome(options=chrome_options)

url = 'https://www.welcometothejungle.com/fr/jobs?refinementList%5Boffices.country_code%5D%5B%5D=FR&refinementList%5Bcontract_type%5D%5B%5D=FULL_TIME&refinementList%5Bcontract_type%5D%5B%5D=TEMPORARY&refinementList%5Bcontract_type%5D%5B%5D=INTERNSHIP&refinementList%5Bcontract_type%5D%5B%5D=APPRENTICESHIP&refinementList%5Bcontract_type%5D%5B%5D=PART_TIME&refinementList%5Bsectors.parent.fr%5D%5B%5D=Tech&refinementList%5Bprofession_name.fr.Business%5D%5B%5D=Business%20Development&refinementList%5Bprofession_name.fr.Business%5D%5B%5D=Commercial&refinementList%5Bprofession_name.fr.Business%5D%5B%5D=Account%20Management&refinementList%5Blanguage%5D%5B%5D=fr&refinementList%5Blanguage%5D%5B%5D=en&refinementList%5Bexperience_level_minimum%5D%5B%5D=0-1&refinementList%5Bexperience_level_minimum%5D%5B%5D=1-3&refinementList%5Bexperience_level_minimum%5D%5B%5D=3-5&query=&page=1'
driver.get(url)

#Charger le modèle NLP et le vectoriseur du modele
model = load('/content/drive/MyDrive/Rocket4Sales/moodel_rocket_nlp.joblib')
vectorizer = load('/content/drive/MyDrive/Rocket4Sales/vectorizer_rocket_nlp.joblib')

#Import des fonctions
def clean_text(text):
    # Conversion en chaîne si ce n'est pas déjà le cas
    text = str(text)
    # Suppression des caractères non alphabétiques
    text = re.sub('[^a-zA-Z\s]', '', text)
    # Conversion en minuscules
    text = text.lower().strip()
    # Suppression des stopwords
    text = ' '.join([word for word in text.split() if word not in stopwords.words('french')])
    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#Liste pour stocker les offres d'emploi
wttj_jobs_offers = []

#Trouver les éléments contenant les annonces et collecter les liens
job_elements = driver.find_elements(By.CSS_SELECTOR, 'div[data-role="jobs:thumb"]')
for job in job_elements:
    job_link = job.find_element(By.CSS_SELECTOR, 'a[href^="/fr/companies"]').get_attribute('href')
    job_ref = job.get_attribute('data-objectid') # Récupération de l'attribut "data-objectid" pour chaque job

    #Ouvrir le lien de l'annonce dans un nouvel onglet
    driver.execute_script("window.open('" + job_link + "');")
    driver.switch_to.window(driver.window_handles[-1])

    # Attendre que la page se charge
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'span.sc-ERObt')))

    # Extraire les informations de l'annonce
    company_name = driver.find_element(By.CSS_SELECTOR, 'span.sc-ERObt').text
    job_title = driver.find_element(By.CSS_SELECTOR, 'h1.sc-ERObt.hzokih.wui-text').text.replace(" hf", "").replace(" fh", "")
    job_location = driver.find_element(By.CSS_SELECTOR, 'p.sc-ERObt.hozLPp').text
    job_type = driver.find_element(By.CSS_SELECTOR, 'div[role="listitem"]').text.replace('\n', '') # Modification ici
    post_date = driver.find_element(By.CSS_SELECTOR, 'time').get_attribute('datetime').replace("\n", "")[:10]
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    job_description_parts = driver.find_elements(By.CSS_SELECTOR, 'div.sc-18ygef-1.ezamTS')
    job_description = ''.join(part.text + '\n' for part in job_description_parts)
    job_description = job_description.replace("Qui sont-ils ?\n", "").replace("Who are they?\n", "").replace("\n", " ")

    wttj_jobs_offers.append({
        "Company name": company_name,
        "jobLink": job_link,
        "jobTitle": job_title,
        "jobLocation": job_location,
        "jobType": job_type,
        "postDate": post_date,
        "jobRef": job_ref,
        "jobDescription": job_description,
        "timestamp": timestamp,
    })

    # Fermer l'onglet et revenir à la page principale
    driver.close()
    driver.switch_to.window(driver.window_handles[0])

# Créer un DataFrame Pandas avec les offres d'emploi
df = pd.DataFrame(wttj_jobs_offers)

# Nettoyage des données (vous pouvez remplacer 'clean_text' par votre fonction de nettoyage)
df['jobTitle'] = df['jobTitle'].apply(clean_text)

# Transformation des données textuelles en utilisant le vectoriseur TF-IDF chargé
X_predict_tfidf = vectorizer.transform(df['jobTitle'])

# Prédiction en utilisant le modèle chargé
df['jobCorrespondance'] = model.predict(X_predict_tfidf)

# Ordre des colonnes
cols = ["Company name", "jobCorrespondance", "jobDescription", "jobLink", "jobLocation", "jobRef", "jobTitle", "jobType", "postDate", "timestamp"]

# Réorganiser les colonnes selon l'ordre spécifié
df = df[cols]

# Chemin du fichier Excel dans votre répertoire local
excel_file_path = '/content/drive/My Drive/Rocket4Sales/jobs_offers.xlsx'

# Enregistrer le DataFrame en tant que fichier Excel
df.to_excel(excel_file_path, index=False)

# Confirmation
print(f'Fichier sauvegardé à {excel_file_path}')

# Fermer le navigateur
driver.quit()

# Afficher les résultats
print(df)

Fichier sauvegardé à /content/drive/My Drive/Rocket4Sales/jobs_offers.xlsx
         Company name    jobCorrespondance  \
0              BRIGAD    Account Executive   
1              MATERA    Account Executive   
2              MATERA               Intern   
3              MATERA               Intern   
4              MATERA               Intern   
5              BRIGAD    Account Executive   
6              BRIGAD    Account Executive   
7              BRIGAD  Key Account Manager   
8              MATERA               Intern   
9              MATERA               Intern   
10          PARTICEEP               Intern   
11            OPENSEE                   IT   
12      LITTLE WORKER    Account Executive   
13      LITTLE WORKER     Sales Operations   
14  EN VOITURE SIMONE               Intern   
15          PARTICEEP               Intern   
16      LITTLE WORKER    Account Executive   
17        GOLDEN BEES      Account Manager   
18            HUNTEED    Account Executive   
19   